Just using this to write and easily test the code for the baseline model. Final implementation will be in a py script, so it can be run from command line using GPU.


# To do!
- create function to extract data to train model
- create function to output tags into appropriate format
- make model - Sort of done
- make sure model works
- train model
- define hyperparamter space and random space search to optimize on dev dataset
- submit results



In [19]:
def convertDataShape(data, vocabulary = {}, labels = [], training = True, paddingToken = '<PAD>'):
    """
    If training is enabled creates a vocabulary of all words in a list. Otherwise, a vocabulary should be passed.
    Does the same with the labels.
    Creates a matrix of sentences and positions, where each value indicates a word via its index in the vocabulary.
    Creates another matrix of sentences and positions, where the values indicate a label.
    '<PAD>' or another user defined token is used as padding for short sentences. Will also act as an unknown token, if not training, it is assumed to be in vocabulary.
    Returns, the vocabulary, the labels and the two matrices.
    
    Input:
    data         - (string list * string list) list - List of sentences. Each sentence is a tuple of two lists. The first is a list of words, the second a list of labels.
    vocabulary   - string : int dictionary          - Dictionary of words in the vocabulary, values are the indices. Should be provided if not training. Defaults to empty dict.
    labels       - string list                      - List of labels to classify. Should be provided if not training. Defaults to empty list.
    training     - boolean                          - Boolean variable deffining whether training is taking place, if yes then a new vocabulary will be created. Defaults to yes.
    paddingToken - string                           - Token to be used as padding and unknown. Default is provided
    
    Output:
    Xmatrix      - 2D np.array                      - 2d numpy array containing the index of the word in the sentence in the vocabulary
    Ymatrix      - 2D np.array                      - 2d numpy array containing the index of the label in the sentence in the labels
    vocabulary   - string : int dictionary          - Dictionary of words, with indices as values, used for training.
    labels       - string list                      - List of all the labels used for classification. (all the labels are expected to be present in the training data, or in other words, the label list provided should be exhaustive)
    """
        
    if training:
        if paddingToken not in vocabulary:
            vocabulary[paddingToken] = 0
        vocabList = list(vocabulary.keys())
        if paddingToken not in labels:
            labels.append(paddingToken)
        for i in data:
            for j in i[0]:
                if j not in vocabulary:
                    vocabList.append(j)
                    vocabulary[j] = vocabList.index(j)
            for j in i[1]:
                if j not in labels:
                    labels.append(j)
                    
    Xmatrix = np.zeros((len(data), max([len(x[0]) for x in data])))
    Ymatrix = np.zeros((len(data), max([len(x[0]) for x in data])))
    
    for i, sentence in zip(range(len(data)), data): #this implementation assumes that the padding Token has value 0 in the vocab dictionary and is in index 0 of the labels list
        for j, word in zip(range(len(sentence[0])), sentence[0]):
            if word in vocabulary:
                Xmatrix[i][j] = vocabulary[word]
            else:
                Xmatrix[i][j] = vocabulary[paddingToken]
        for j, label in zip(range(len(sentence[1])), sentence[1]):
            Ymatrix[i][j] = labels.index(label)
    
    return Xmatrix, Ymatrix, vocabulary, labels

In [13]:
import numpy as np
import torch
import TorchCRF
from torch import nn

#Creating the class for the baseline Model

class baselineModel(torch.nn.Module):

    def __init__(self, nWords, tags, dimEmbed, dimHidden):
        super().__init__()
        self.dimEmbed = dimEmbed
        self.dimHidden = dimHidden
        self.vocabSize = nWords
        self.tagSetSize = len(tags)
        self.tagSet = tags

        self.embed = nn.Embedding(nWords, dimEmbed)
        self.LSTM = nn.LSTM(dimEmbed, dimHidden, batch_first = True, bidirectional = True)
        self.linear = nn.Linear(dimHidden * 2, self.tagSetSize) 
        
        self.CRF = TorchCRF.CRF(self.tagSetSize, batch_first = True)


        
    def forwardTrain(self, inputData):
        wordVectors = self.embed(inputData)
        out, _ = self.LSTM(wordVectors.view((inputData.shape[0], inputData.shape[1], self.dimEmbed)))
        out = self.linear(out)
        preds = self.CRF.decode(out)
        loss = self.CRF.forward(out, preds)
        return loss
        
        
    def forwardPred(self, inputData):
        wordVectors = self.embed(inputData)
        out, _ = self.LSTM(wordVectors.view((inputData.shape[0], inputData.shape[1], self.dimEmbed)))
        out = self.linear(out)
        preds = self.CRF.decode(out)
        return preds


In [23]:
# two first sentences of EWT training dataset so that quickdebugging can be run

tags = ["O", "B-PER", "I-PER", "B-LOC", "I-LOC", "B-ORG", "I-ORG"]

trainingDebugSen = [["Where", "in", "the", "world", "is", "Iguazu", "?"], ["Iguazu", "Falls"]]
trainingDebugTags = [["O", "O", "O", "O", "O", "B-LOC", "O"], ["B-LOC", "I-LOC"]]

data, labels, vocab, tags = convertDataShape(list(zip(trainingDebugSen, trainingDebugTags)))

In [26]:


DIM_EMBEDDING = 100
LSTM_HIDDEN = 50
BATCH_SIZE = 1
LEARNING_RATE = 0.01
EPOCHS = 5

model = baselineModel(len(vocab), tags, DIM_EMBEDDING, LSTM_HIDDEN)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
    model.train()
    
    model.zero_grad()
     
    optimizer.zero_grad()
    loss = model.forwardTrain(torch.tensor(data, dtype=torch.long))
        
        
    loss.backward()
    optimizer.step()
        
    
    print(epoch)
        



D:\Anaconda\Lib\site-packages\TorchCRF\__init__.py:305: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ..\aten\src\ATen\native\TensorCompare.cpp:519.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


AttributeError: 'list' object has no attribute 'shape'